In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import tensorflow as tf
import keras

device = "/GPU:0" if tf.config.list_physical_devices("GPU") else "/CPU:0"
print(device)

/CPU:0


In [2]:
# Load data
x = np.load('../../../data/CIFAR_10/x.npy')
y = np.load('../../../data/CIFAR_10/y.npy')

In [3]:
# Normalize pixel values
x = x / 255.0

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=True, random_state=42
)

In [4]:
# Define the model
input_shape = (32, 32, 3)
n_classes = 10

model = keras.Sequential([
    keras.layers.Input(shape=input_shape),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation=keras.layers.LeakyReLU()),
    keras.layers.Dense(512, activation=keras.layers.LeakyReLU()),
    keras.layers.Dense(n_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
with tf.device(device):
    model.fit(x_train, y_train, epochs=10, batch_size=64)

In [ ]:
# Evaluate model
with tf.device(device):
    y_pred = model.predict(x_test)
    y_pred = np.argmax(y_pred, axis=1)

metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_score": f1_score(y_test, y_pred, average="weighted"),
    "precision": precision_score(y_test, y_pred, average="weighted"),
    "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
}
print(metrics)